In [1]:
from sklearn.utils import shuffle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [2]:
train_dataset = "train.csv"
df_train = shuffle(pd.read_csv(train_dataset))
x_cols_to_delete = ["ID", "Extracellular","Nucleus","Cytoplasm","Membrane","Cell membrane"]
x_train = df_train
for col in x_cols_to_delete:
    x_train = x_train.drop(col,axis=1)
x_train = x_train.fillna("")

y_cols_to_delete = ["ID", "sequence", "Kingdom", "seq_len"]
y_train = df_train
for col in y_cols_to_delete:
    y_train = y_train.drop(col,axis=1)
y_train = y_train.fillna("")

In [3]:
test_dataset = "test.csv"
df_test = pd.read_csv(test_dataset)
x_test = df_test.drop("ID",axis=1)
x_test = x_test.fillna("")

In [4]:
vectorizer = TfidfVectorizer(ngram_range=(1, 4), analyzer='char')
vectorizer.fit(list(x_train['sequence']))
train_vectorized = vectorizer.transform(x_train['sequence'])
test_vectorized = vectorizer.transform(x_test['sequence'])

XGBoost ML algorithm.

In [ ]:
import xgboost as xgb
import sklearn
import numpy as np
from sklearn.model_selection import GridSearchCV

def modelfit(alg, train_data_features, train_labels,useTrainCV=True, cv_folds=5):

    if useTrainCV:
        params=alg.get_xgb_params()
        xgb_param=dict([(key,[params[key]]) for key in params])

        boost = xgb.sklearn.XGBClassifier()
        cvresult = GridSearchCV(boost,xgb_param,cv=cv_folds)
        cvresult.fit(train_vectorized,y_train)
        alg=cvresult.best_estimator_


    #Fit the algorithm on the data
    alg.fit(train_data_features, train_labels)

    #Predict training set:
    dtrain_predictions = alg.predict(train_data_features)
    dtrain_predprob = alg.predict_proba(train_data_features)[:,1]

    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % sklearn.metrics.accuracy_score(train_labels, dtrain_predictions))

xgb1 = xgb.sklearn.XGBClassifier(
 learning_rate =0.1,
 n_estimators=50,
 max_depth=3,
 num_class=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective='multi:softmax')    


modelfit(xgb1, train_vectorized, y_train)

In [14]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0,verbose=1)
model.fit(train_vectorized,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.5min finished


RandomForestClassifier(random_state=0, verbose=1)

In [15]:
result = model.predict_proba(test_vectorized)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished


In [16]:
max_result = []
for col in result:
    new_col=[]
    for row in col:
        new_col.append(round(row[0],2))
    max_result.append(new_col)
result = max_result


In [17]:
import csv

In [18]:
csvfile = open("submit.csv",'w',encoding='utf-8')
csvfile_writer = csv.writer(csvfile)


In [19]:
columns = ["ID", "target"]

In [20]:
csvfile_writer.writerow(columns)

11

In [12]:
columns = ["Nucleus","Cytoplasm","Membrane","Cell membrane","Extracellular"]
for i_col,col in enumerate(columns):
    for i_row in range(len(result[0])):
        csv_line = [f"test_{i_row}_{col}",f"{result[i_col][i_row]}"]
        csvfile_writer.writerow(csv_line)

In [13]:
csvfile.close()

In [112]:
rows = [None]*len(result[0])
for i in range(len(result[0])):
    rows[i] = [col[i] for col in result]


In [118]:
for i_row in range(len(rows)):
    for i_col in range(len(rows[0])):
        rows[i_row][i_col] = np.max(rows[i_row][i_col])
print(rows[0])

[0.621721222631693, 0.5745718621287665, 0.751680034684587, 0.877411662692391, 0.9167569911120745]
